# Кратко об этом исследовании
Данные из Остинского центра животных, то есть приюта, -- с 1 октября 2013 по **март 2016**. 

### Цель

Требуется предсказать судьбу каждого животного по данным о нём сведениям. По сути, обычная задача категоризации. Классы: Adoption, Died, Euthanasia, Return to owner, Transfer. 

Все классы считаем одинаково важными все зависимости от представленности в выборке. Качество предсказаний оценивается поэтому с помощью macro-averaged F1 score.

---

**Задание**

Пользоваться в точности предложенной в этом шаблоне схемой необязательно, но в этом ноутбуке должен быть 
- внятный и чистый, 
- прокомментированный, 
- воспроизводимый (зафиксируйте все random seeds, которые возможно),
- мотивированный
**код**, который **генерирует ваше лучшее решение**.

А также пока **неформальный отчёт** о проделанной работе.


### Методы

`TODO: Напишите, как пробовали предобрабатывать признаки`

Постарался сразу выделить все признаки, что можно было, и не возвращаться. Разделил животных на тех, у кого есть имя и у кого нет. Неизвестный срок заполнил средним значением. Объединил мало встречающиеся редкие породы и цвета. Пробовал выделять щенков и котят в отдельный признак - результата не дало, хотя было бы логично, возможно, что-то сделал не так. Стерилизованных животных тоже объединил.


`TODO:Напишите, какие модели и с какими параметрами вы пробовали`

Попробовал две модели, предложенную DecisionTreeClassifier и XGBClassifier от XGBoost.

Тактика выбора параметров у меня одна: сначала берем широкий диапазон параметров, смотрим максимум, а дальше наблюдаем, какие изменения ведут к улучшению скора, пытаемся их оптимизировать. Иногда направлений движения несколько, смотрим все. В DecisionTreeClassifier заметил, что любые параметры max_features вели к ухудшению скора. Этот результат сохраняется в submission1.csv.

`Cross-validation f1 macro: 0.422 (+/- 0.021) for 'criterion': 'entropy', 'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'`

В XGBClassifier тактика такая же. Здесь я усмотрел два направления движения, которые давали похожий результат.
Первый это увеличивать n_estimators, увеличивать max_depth и добавлять регуляризацию в виде gamma, reg_alpha. Такой путь занимает больше времени на обучение. Второй это иметь маленький n_estimators и max_depth пониже c незначительной регуляризацией. Модель быстро учится и даже дает чуть лучший результат в итоге. Оба варианта я оставил в коде, закоментив первый. Этот вериант сохраняется в submission.csv.

Первый вариант:  
`Cross-validation f1 macro: 0.448 (+/- 0.024) for 'gamma': 0.04, 'learning_rate': 0.3, 'max_depth': 13, 'n_estimators': 300, 'reg_alpha': 0.05`

Второй вариант:  
`Cross-validation f1 macro: 0.455 (+/- 0.020) for 'gamma': 0.005, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 50, 'reg_alpha': 0.01`

Еще была идея попробовтать RandomForest, LightBM (но кажется это примерно то же самое, что и XGBClassifier), а потом ввести голосование по всем моделям, но руки не дошли.


### Результаты

`TODO: Поделитесь наблюдениями, историями успеха и зря потраченными усилиями; что интересного можете сказать о наборе данных? какие выводы?`

Больше времени я потратил на обучение XGBClassifier, потому что подумал, что она более сложная и должна выдавать лучший результат, так и получилось. В какой-то момент возникло ощущение, что у меня появился потолок в скоре, который я обьяснял себе переобучением, но введение регуляризации этому не особо помогло. Зря потраченными усилиями оказалось увеличение параметров и долгое обучение XGBClassifier. Сравнительное низкие простые параметры все равно показывали лучший результат, видимо, переобучение. Вывод, что иногда лучше понизить сложность модели, чем вводить регуляризацию. По самим данным возникло ощущение, что их довольно мало для каких-то сложных моделей.

---
---
---

## Конфиги и константы
(пожалуйста, без волшебных чисел в коде)


In [3]:
OUTCOME2LABEL = {"Adoption" : 0, 
                 "Transfer": 1, 
                 "Return_to_owner": 2, 
                 "Euthanasia": 3, 
                 "Died": 4
                }
LABEL2OUTCOME = {v: k for k,v in OUTCOME2LABEL.items()}
FOLD_K = 4


## Библиотеки
(все импорты желательно должны быть здесь)

In [5]:
import sklearn
sklearn.__version__

'1.4.2'

In [6]:
#pip install xgboost 
# Нужно установить XGBoost, я его использовал

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost
import warnings
warnings.filterwarnings('ignore')


from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from xgboost import XGBClassifier, plot_importance, plot_tree

In [8]:
train_data = pd.read_csv("train.csv", encoding="utf-8")
test_data = pd.read_csv("test.csv", encoding="utf-8")

# Объединим трейн и тест, чтобы вместе их обработать
train_data['IsTrain'] = 1
test_data['IsTrain'] = 0

test_data['Outcome'] = np.nan

combined_data = pd.concat([train_data, test_data], ignore_index=True)

print(combined_data.isnull().sum())
print(combined_data.head())

Name              7691
SexuponOutcome       1
AnimalType           0
AgeuponOutcome      18
Breed                0
Color                0
DateTime             0
Outcome           8019
ID                   0
IsTrain              0
dtype: int64
      Name SexuponOutcome AnimalType AgeuponOutcome                    Breed  \
0    Socks  Neutered Male        Cat       2 months   Domestic Shorthair Mix   
1     Vera  Intact Female        Cat        1 month   Domestic Shorthair Mix   
2  Biscuit  Neutered Male        Dog       3 months  Chihuahua Shorthair Mix   
3   Kitten  Spayed Female        Cat        2 years   Domestic Shorthair Mix   
4      NaN  Neutered Male        Cat       2 months   Domestic Shorthair Mix   

          Color             DateTime  Outcome  ID  IsTrain  
0   Black/White  2014-06-11 14:36:00      0.0   0        1  
1  Tortie/White  2014-07-18 08:10:00      3.0   1        1  
2        Yellow  2016-01-02 17:28:00      2.0   2        1  
3        Calico  2014-02-19 17:2

### Подготовка признаков

#### Даты

In [10]:
def pandas_dates2number(date_series: pd.Series):
    return pd.to_datetime(date_series).values.astype(np.int64) // 10 ** 9

combined_data['DateTime'] = pandas_dates2number(combined_data['DateTime'])

#### Имена


In [12]:
combined_data['HasName'] = (~combined_data['Name'].isnull()).astype(int) # Есть имя или нет
combined_data.drop('Name', axis=1, inplace=True)
print(combined_data.head())

  SexuponOutcome AnimalType AgeuponOutcome                    Breed  \
0  Neutered Male        Cat       2 months   Domestic Shorthair Mix   
1  Intact Female        Cat        1 month   Domestic Shorthair Mix   
2  Neutered Male        Dog       3 months  Chihuahua Shorthair Mix   
3  Spayed Female        Cat        2 years   Domestic Shorthair Mix   
4  Neutered Male        Cat       2 months   Domestic Shorthair Mix   

          Color    DateTime  Outcome  ID  IsTrain  HasName  
0   Black/White  1402497360      0.0   0        1        1  
1  Tortie/White  1405671000      3.0   1        1        1  
2        Yellow  1451755680      2.0   2        1        1  
3        Calico  1392830820      0.0   3        1        1  
4  Orange Tabby  1405964040      0.0   4        1        0  


#### Возраст


In [14]:
def convert_age_to_days(age_str):
    if age_str == 'Unknown' or pd.isnull(age_str):
        return np.nan
    age_num, age_unit = age_str.split()
    age_num = float(age_num)
    if 'year' in age_unit:
        return age_num * 365
    elif 'month' in age_unit:
        return age_num * 30
    elif 'week' in age_unit:
        return age_num * 7
    elif 'day' in age_unit:
        return age_num
    else:
        return np.nan
# Сделаем возраст числом
combined_data['AgeInDays'] = combined_data['AgeuponOutcome'].apply(convert_age_to_days)
# Заполним средним отсутсвующие данные о сроке
median_age = combined_data[combined_data['IsTrain'] == 1]['AgeInDays'].median() 
combined_data['AgeInDays'].fillna(median_age, inplace=True)
combined_data.drop('AgeuponOutcome', axis=1, inplace=True)


print(combined_data.head())

  SexuponOutcome AnimalType                    Breed         Color  \
0  Neutered Male        Cat   Domestic Shorthair Mix   Black/White   
1  Intact Female        Cat   Domestic Shorthair Mix  Tortie/White   
2  Neutered Male        Dog  Chihuahua Shorthair Mix        Yellow   
3  Spayed Female        Cat   Domestic Shorthair Mix        Calico   
4  Neutered Male        Cat   Domestic Shorthair Mix  Orange Tabby   

     DateTime  Outcome  ID  IsTrain  HasName  AgeInDays  
0  1402497360      0.0   0        1        1       60.0  
1  1405671000      3.0   1        1        1       30.0  
2  1451755680      2.0   2        1        1       90.0  
3  1392830820      0.0   3        1        1      730.0  
4  1405964040      0.0   4        1        0       60.0  


#### Цвета и породы

Повторяющихся категорий много, так что можно закодировать их и так, однако **некоторые из них можно и растащить на части**

Необязательно, но можете попробовать, вдруг поможет :)

In [16]:
# Выделим смешанные породы, разделим их. Редкие бороды объединим
combined_data['IsMix'] = combined_data['Breed'].str.contains('Mix|/', regex=True).astype(int)
combined_data['PrimaryBreed'] = combined_data['Breed'].str.split('/').str[0]
combined_data['PrimaryBreed'] = combined_data['PrimaryBreed'].str.replace(' Mix', '', regex=False)

breed_counts = combined_data[combined_data['IsTrain'] == 1]['PrimaryBreed'].value_counts()

rare_breeds = breed_counts[breed_counts < 15].index
combined_data['PrimaryBreed'] = combined_data['PrimaryBreed'].replace(rare_breeds, 'Other')
combined_data.drop('Breed', axis=1, inplace=True)

print(combined_data[combined_data['IsTrain'] == 1]['PrimaryBreed'].value_counts())

PrimaryBreed
Domestic Shorthair        6242
Chihuahua Shorthair       1495
Pit Bull                  1471
Labrador Retriever        1375
Domestic Medium Hair       622
                          ... 
Papillon                    15
Vizsla                      15
Harrier                     15
Shiba Inu                   15
Parson Russell Terrier      15
Name: count, Length: 89, dtype: int64


In [17]:
# Обработаем цвета, объединим редкие
combined_data['NumColors'] = combined_data['Color'].str.count('/') + 1
combined_data['PrimaryColor'] = combined_data['Color'].str.split('/').str[0]
color_counts = combined_data[combined_data['IsTrain'] == 1]['PrimaryColor'].value_counts()
rare_colors = color_counts[color_counts < 10].index
combined_data['PrimaryColor'] = combined_data['PrimaryColor'].replace(rare_colors, 'Other')

print(combined_data[combined_data['IsTrain'] == 1]['PrimaryColor'].value_counts())

combined_data.drop('Color', axis=1, inplace=True)

PrimaryColor
Black              4491
White              2350
Brown Tabby        1813
Brown              1358
Tan                1173
Orange Tabby        911
Blue                820
Tricolor            572
Red                 557
Blue Tabby          476
Brown Brindle       473
Tortie              409
Calico              383
Chocolate           321
Torbie              276
Sable               215
Buff                199
Cream Tabby         179
Cream               172
Yellow              156
Gray                155
Fawn                145
Lynx Point          124
Seal Point          119
Blue Merle          115
Black Brindle        66
Flame Point          64
Gold                 56
Black Smoke          50
Brown Merle          49
Black Tabby          44
Gray Tabby           38
Red Merle            36
Silver               35
Silver Tabby         32
Blue Tick            29
Orange               27
Lilac Point          24
Yellow Brindle       23
Blue Point           23
Red Tick             23
Tor

#### Пол

Пола, как мы видим, четыре: стерилизованные и нестерилизованные самки и самцы. Также пол может быть неизвестен.

Может, факт стерилизации стоит сделать отдельным признаком? Но это неточно.

In [19]:
# Заполним отстуствующие данные пола
print(combined_data.isnull().sum())
combined_data['SexuponOutcome'].fillna('Unknown', inplace=True)

# Объединим стерилизованных
def simplify_sex(sex):
    if sex in ['Neutered Male', 'Spayed Female']:
        return 'Neutered/Spayed'
    elif sex in ['Intact Male', 'Intact Female']:
        return 'Intact'
    else:
        return 'Unknown'

combined_data['Sex'] = combined_data['SexuponOutcome'].apply(simplify_sex)
combined_data.drop('SexuponOutcome', axis=1, inplace=True)

print(combined_data)

SexuponOutcome       1
AnimalType           0
DateTime             0
Outcome           8019
ID                   0
IsTrain              0
HasName              0
AgeInDays            0
IsMix                0
PrimaryBreed         0
NumColors            0
PrimaryColor         0
dtype: int64
      AnimalType    DateTime  Outcome    ID  IsTrain  HasName  AgeInDays  \
0            Cat  1402497360      0.0     0        1        1       60.0   
1            Cat  1405671000      3.0     1        1        1       30.0   
2            Dog  1451755680      2.0     2        1        1       90.0   
3            Cat  1392830820      0.0     3        1        1      730.0   
4            Cat  1405964040      0.0     4        1        0       60.0   
...          ...         ...      ...   ...      ...      ...        ...   
26724        Dog  1428773400      NaN  8014        0        1      240.0   
26725        Dog  1444659360      NaN  8015        0        1     3285.0   
26726        Dog  141883350

In [20]:
categorical_features = ['AnimalType', 'Sex', 'PrimaryBreed', 'PrimaryColor']
combined_data = pd.get_dummies(combined_data, columns=categorical_features) # Воспользуемся простым методом

train_data_processed = combined_data[combined_data['IsTrain'] == 1].drop('IsTrain', axis=1)
test_data_processed = combined_data[combined_data['IsTrain'] == 0].drop(['IsTrain', 'Outcome'], axis=1)

In [21]:
X_train = train_data_processed.drop(['Outcome', 'ID'], axis=1)
y_train = train_data_processed['Outcome'].astype(int)
X_test = test_data_processed.drop('ID', axis=1)

#### DecisionTreeClassifier

In [23]:
# Выбираем параметры. Тактика у меня одна: сначала берем широкий диапазон параметров, смотрим максимум, а дальше наблюдаем, 
# какие изменения ведут к улучшению скора, пытаемся их оптимизировать. Иногда направлений движения несколько, смотрим все.
# Самая широкая сетка параметров обучается долго, ее я здесь не оставил, тут уже подобранные.
# Заметил, что любые параметры max_features вели к ухудшению скора
# В данной модели максимум, что я смог получить, это 0.42.
param_grid = [{
    'min_samples_leaf': [1, 3, 5],
    'min_samples_split': [2, 5],
    'max_depth': [20, 18, 15],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
     }]

# Поиск по заданной решётке параметров
clf = GridSearchCV(tree.DecisionTreeClassifier(random_state=100, class_weight="balanced"),
                       param_grid = param_grid, 
                       scoring='f1_macro', 
                       verbose=1, 
                       cv=FOLD_K) 

# Запускаем поиск
clf.fit(X_train, y_train)

print("Best params on dev set:")
print(clf.best_params_)
    
print("Scores on development set:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    
# Обучаем на всём с "лучшими" параметрами
best_estimator = clf.best_estimator_
best_estimator.fit(X_train, y_train)
    
# Порождаем и сохраняем сабмит
y_pred = best_estimator.predict(X_test)
submission = pd.DataFrame({
    "ID": test_data["ID"],
    "Outcome": y_pred
})

submission_file_name = f"submission1.csv"
submission.to_csv(submission_file_name, index=False)

# Кросс-валидация
cv_scores = cross_val_score(
    estimator=best_estimator,
    X=X_train,
    y=y_train,
    cv=FOLD_K,
    scoring='f1_macro'
)
print(f"Cross-validation f1 macro: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

Fitting 4 folds for each of 72 candidates, totalling 288 fits
Best params on dev set:
{'criterion': 'entropy', 'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
Scores on development set:
0.411 (+/-0.020) for {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
0.411 (+/-0.018) for {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}
0.404 (+/-0.017) for {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}
0.402 (+/-0.020) for {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'random'}
0.396 (+/-0.012) for {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 2, 'splitter': 'best'}
0.391 (+/-0.009) for {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 2, 'splitter': 'random'}
0.396 (+/-

#### XGBClassifier

In [25]:
xgb_model = XGBClassifier(
    objective='multi:softprob',
    num_class=len(np.unique(y_train)),
    eval_metric='mlogloss',
    random_state=42,
)

# Тактика выбора параметров такая же. Здесь я усмотрел два направления движения, которые давали похожий результат.
# Первый это увеличивать n_estimators, повышать max_depth и добавлять регуляризацию в виде gamma, reg_alpha. Такой вариант тратит больше времени на обучение.
# Второй это иметь маленький n_estimators и max_depth пониже c незначительной регуляризацией (Оба варианта я оставил, первый из них закоментил). 

'''
# Первый вариант
# Cross-validation f1 macro: 0.448 (+/- 0.024) for {'gamma': 0.04, 'learning_rate': 0.3, 'max_depth': 13, 'n_estimators': 300, 'reg_alpha': 0.05}
param_grid = {
    'n_estimators': [300],
    'learning_rate': [0.3, 0.01],
    'max_depth': [5, 13, 20],
    'gamma': [0.04, 0.001],
    'reg_alpha': [0.05, 0.001],
}

'''
# Второй вариант
# Cross-validation f1 macro: 0.455 (+/- 0.020) for {'gamma': 0.005, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 50, 'reg_alpha': 0.01}
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.1, 0.3],
    'max_depth': [5, 11, 20],
    'gamma': [0.005],
    'reg_alpha': [0.01],
}


skf = StratifiedKFold(n_splits=FOLD_K, shuffle=True, random_state=42)

# Поиск по заданной решётке параметров
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=skf,
    verbose=1,
    n_jobs=-1
)

# Запускаем поиск
grid_search.fit(X_train, y_train)

print("Best parameters found on development set:")
print(grid_search.best_params_)
print()

print("Grid scores on development set:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f +- %0.3f for %r" % (mean, std * 2, params))

# Обучаем на всём с "лучшими" параметрами
best_model = grid_search.best_estimator_

# Порождаем и сохраняем сабмит
X_test = test_data_processed.drop('ID', axis=1)
y_pred = best_model.predict(X_test)

submission = pd.DataFrame({
    'ID': test_data['ID'],
    'Outcome': y_pred  
})

submission.to_csv('submission.csv', index=False)

# Кросс-валидация
cv_scores = cross_val_score(
    estimator=best_model,
    X=X_train,
    y=y_train,
    cv=FOLD_K,
    scoring='f1_macro'
)
print(f"Cross-validation f1 macro: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

Fitting 4 folds for each of 12 candidates, totalling 48 fits
Best parameters found on development set:
{'gamma': 0.005, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 50, 'reg_alpha': 0.01}

Grid scores on development set:
0.426 +- 0.016 for {'gamma': 0.005, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'reg_alpha': 0.01}
0.432 +- 0.017 for {'gamma': 0.005, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 0.01}
0.442 +- 0.020 for {'gamma': 0.005, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 50, 'reg_alpha': 0.01}
0.443 +- 0.011 for {'gamma': 0.005, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'reg_alpha': 0.01}
0.439 +- 0.018 for {'gamma': 0.005, 'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 50, 'reg_alpha': 0.01}
0.439 +- 0.006 for {'gamma': 0.005, 'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 100, 'reg_alpha': 0.01}
0.432 +- 0.018 for {'gamma': 0.005, 'learning_rate': 0.3, 'max_depth': 5, 'n_estimat